Chapter 4. PhD Thesis. Andra Sonea.

Example of script for extracting bank branches using the Branch Location Open Banking API. The structure of the API is the same for all the CMA9 banks however the precise way the API is called by the developer, the branch numbering convention or its absence differ between banks. Other differences between banks: 

1) the frequency of updating the data (visible in the header of the API output, field LastUpdated). 
2) discrepancies between the API output and their website. These should be the same (i.e. a branch shoudl appear both on the website and in the API with the same location and operating schedule)
3) for some banks the mobile branches location (Latitude & Longitude is missing or continuously changing)
4) some outputs result in random formatting problems

During this research I have interacted with some of the banks in regards with the errors of thier API output which made the API unusable. Some of these errors have been corrected.

In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
import http.client
import json

#### CHANGE THE MONTH!!!

In [3]:
period = '2024_07'
bank = 'LBG'

In [4]:
input_file_json = '../../../output/'+ period + '_Branches/' + bank + '_'+ period +'.json'

output_file_json = '../../../output/'+ period + '_Branches/' + bank + '_'+ period +'.json'

output_file_csv = '../../../output/'+ period + '_Branches/'+ bank + '_'+ period + '.csv'

input_file_csv= '../../../output/'+ period + '_Branches/'+ bank + '_'+ period + '_capacity.csv'

In [5]:
#call the API
conn = http.client.HTTPSConnection("api.lloydsbank.com")

In [6]:
headers = {
    'if-modified-since': "REPLACE_THIS_VALUE",
    'if-none-match': "REPLACE_THIS_VALUE",
    'accept': "application/prs.openbanking.opendata.v2.2+json"
    }

conn.request("GET", "/open-banking/v2.2/branches", headers=headers)
conn

In [7]:
res = conn.getresponse()
res

In [8]:
data_raw = res.read()
data_raw

b'{\n    "meta": {\n        "LastUpdated": "2024-06-24T08:30:14.480Z",\n        "TotalResults": 547,\n        "Agreement": "Use of the APIs and any related data will be subject to the terms of the Open Licence and subject to terms and conditions",\n        "License": "https://www.openbanking.org.uk/open-licence",\n        "TermsOfUse": "https://www.openbanking.org.uk/terms-conditions"\n    },\n    "data": [\n        {\n            "Brand": [\n                {\n                    "BrandName": "Lloyds Bank",\n                    "Branch": [\n                        {\n                            "Identification": "30109100",\n                            "SequenceNumber": "00",\n                            "Name": "Bishopsworth",\n                            "Type": "Physical",\n                            "CustomerSegment": [\n                                "Personal",\n                                "Business"\n                            ],\n                            "Availabilit

In [9]:
data = json.loads(data_raw.decode('utf-8'), encoding='utf-8')
data

{'meta': {'LastUpdated': '2024-06-24T08:30:14.480Z',
  'TotalResults': 547,
  'Agreement': 'Use of the APIs and any related data will be subject to the terms of the Open Licence and subject to terms and conditions',
  'License': 'https://www.openbanking.org.uk/open-licence',
  'TermsOfUse': 'https://www.openbanking.org.uk/terms-conditions'},
 'data': [{'Brand': [{'BrandName': 'Lloyds Bank',
     'Branch': [{'Identification': '30109100',
       'SequenceNumber': '00',
       'Name': 'Bishopsworth',
       'Type': 'Physical',
       'CustomerSegment': ['Personal', 'Business'],
       'Availability': {'StandardAvailability': {'Day': [{'Name': 'Monday',
           'OpeningHours': [{'OpeningTime': '09:30', 'ClosingTime': '16:30'}]},
          {'Name': 'Tuesday',
           'OpeningHours': [{'OpeningTime': '09:30', 'ClosingTime': '16:30'}]},
          {'Name': 'Wednesday',
           'OpeningHours': [{'OpeningTime': '09:30', 'ClosingTime': '16:30'}]},
          {'Name': 'Thursday',
         

In [10]:
with open(output_file_json, 'w') as f:
    json.dump(data, f)

In [11]:
with open(input_file_json, "r") as read_file:
    data = json.load(read_file)

In [12]:
f = open(output_file_csv, "w")

f.write('Brand,ID,Type,Branch Name,Town,Postcode,Lat,Long,MonOp,MonCls,TueOp,TueCls,WedOp,WedCls,ThuOp,ThuCls,FriOp,FriCls,SatOp,SatCls,SunOp,SunCls\n')

for brand in data['data'][0]['Brand']:
    for branch in brand['Branch']:
        ID = branch['Identification']
        Btype =  branch ['Type']
        BName = branch['Name']
        town = branch ['PostalAddress'].get('TownName', 'None')
        postcode = branch['PostalAddress']['PostCode']
        try:
            latitude = float(branch ['PostalAddress']['GeoLocation']['GeographicCoordinates']['Latitude'])
            longitude = float(branch['PostalAddress']['GeoLocation']['GeographicCoordinates']['Longitude'])
        except KeyError:
            latitude = 0
            longitude = 0
            
        days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        opening = {day: {'Open':'', 'Close': ''} for day in days}
        try:
            availability = branch['Availability']['StandardAvailability']['Day']
            for day in availability:
                opening[day['Name']]['Open'] = day['OpeningHours'][0]['OpeningTime']
                opening[day['Name']]['Close'] = day['OpeningHours'][0]['ClosingTime']
        except KeyError:
            pass
        f.write('{0},{1},{2},{3},{4},{5},{6:6f},{7:6f}'.format(brand['BrandName'], ID, Btype, BName, town, postcode, latitude, longitude))
        for day in days:
            f.write(',{0},{1}'.format(opening[day]['Open'],opening[day]['Close']))
        f.write('\n')
f.close()

The branches data is not always correctly formatted. For one or two records the Postcode will appear in the latutide field. As there are not rules, better to correct the formatting manually. Afterwards, re-start the script from here.

- for 2023/01 two branches were wrongly formatted and I copied their Longitude from the previous period

In [13]:
branches = pd.read_csv(output_file_csv)

In [14]:
branches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 547 entries, 0 to 546
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Brand        547 non-null    object 
 1   ID           547 non-null    int64  
 2   Type         547 non-null    object 
 3   Branch Name  547 non-null    object 
 4   Town         547 non-null    object 
 5   Postcode     547 non-null    object 
 6   Lat          547 non-null    float64
 7   Long         547 non-null    float64
 8   MonOp        545 non-null    object 
 9   MonCls       545 non-null    object 
 10  TueOp        535 non-null    object 
 11  TueCls       535 non-null    object 
 12  WedOp        536 non-null    object 
 13  WedCls       536 non-null    object 
 14  ThuOp        542 non-null    object 
 15  ThuCls       542 non-null    object 
 16  FriOp        534 non-null    object 
 17  FriCls       534 non-null    object 
 18  SatOp        299 non-null    object 
 19  SatCls  

In [15]:
branches['Lat'] = branches['Lat'].astype(float)

In [16]:
#calculate total open time (in minutes), on a Monday
branches['MonDuration']=(pd.to_datetime(branches['MonCls']) - pd.to_datetime(branches['MonOp']))/np.timedelta64(1,'m')

In [17]:
#calculate total open time (in minutes), on a Tuesday
branches['TueDuration']=(pd.to_datetime(branches['TueCls']) - pd.to_datetime(branches['TueOp']))/np.timedelta64(1,'m')

In [18]:
#calculate total open time (in minutes), on a Wednesday
branches['WedDuration']=(pd.to_datetime(branches['WedCls']) - pd.to_datetime(branches['WedOp']))/np.timedelta64(1,'m')

In [19]:
#calculate total open time (in minutes), on a Thursday
branches['ThuDuration']=(pd.to_datetime(branches['ThuCls']) - pd.to_datetime(branches['ThuOp']))/np.timedelta64(1,'m')

In [20]:
#calculate total open time (in minutes), on a Friday
branches['FriDuration']=(pd.to_datetime(branches['FriCls']) - pd.to_datetime(branches['FriOp']))/np.timedelta64(1,'m')

In [21]:
#calculate total open time (in minutes), on a Saturday
branches['SatDuration']=(pd.to_datetime(branches['SatCls']) - pd.to_datetime(branches['SatOp']))/np.timedelta64(1,'m')

In [22]:
#calculate total open time (in minutes), on a Sunday
branches['SunDuration']=(pd.to_datetime(branches['SunCls']) - pd.to_datetime(branches['SunOp']))/np.timedelta64(1,'m')

In [23]:
branches['MonDuration'] = branches['MonDuration'].fillna(0)
branches['TueDuration'] = branches['TueDuration'].fillna(0)
branches['WedDuration'] = branches['WedDuration'].fillna(0)
branches['ThuDuration'] = branches['ThuDuration'].fillna(0)
branches['FriDuration'] = branches['FriDuration'].fillna(0)
branches['SatDuration'] = branches['SatDuration'].fillna(0)
branches['SunDuration'] = branches['SunDuration'].fillna(0)

In [24]:
branches['TotalWkOp']= branches['MonDuration']+branches['TueDuration']+ \
                        branches['WedDuration']+branches['ThuDuration']+ \
                        branches['FriDuration']+branches['SatDuration']+branches['SunDuration']

Calculation of Capacity where the 2790min/week was the assumed "normal branch" in 2019. This threshold was abandoned later on in the thesis because the typology and the schedules of bank branches started varying much more. There is no "normal branch" anymore.


In [25]:
# Capacity
branches['Capacity']= branches['TotalWkOp']/2790

In [26]:
# Save points file as csv
branches.to_csv(input_file_csv)